In [2]:
import numpy as np
import math
from scipy import optimize

In [2]:
def f(x):
    return 4*x[0]**2 + 3*x[0]*x[1] + 3*x[1]**2 + 2*x[0] + 3*x[1] + 3


def grad(x):
    return np.array([
        8*x[0]+3*x[1]+2,
        6*x[1]+3*x[0]+3,
    ])


def hessiano(x):
    return np.array([
        [8, 3],
        [3, 6]
    ])


In [3]:
def gradienteConjugado(x0, b, k=0, tol=1e-6):
    r = grad(x0)
    p = r*-1
    print(x0)
    print("x0, f(x^k), aMD, b")
    rDotr = np.dot(r, r)
    AdotP = np.dot(hessiano(x0), p)
    while np.linalg.norm(grad(x0)) >= tol:
        alpha = rDotr / np.dot(AdotP, p)
        x0 = x0 + alpha*p
        r1 = r + alpha * AdotP
        b = (np.dot(r1, r1))/rDotr
        p = -r1 + b*p
        print(x0, f(x0), alpha, b)
        r = r1
        rDotr = np.dot(r, r)
        AdotP = np.dot(hessiano(x0), p)
    return x0


In [4]:
def newtonNDim(x0, tol=1e-4):
    k = 0
    while np.linalg.norm(grad(x0)) > tol:
        x0 = x0 - np.dot(np.linalg.inv(hessiano(x0)), grad(x0))
        k = k+1
        print(k, f(x0), x0)
    return x0


In [12]:
x0 = np.array([0, 0])
b = np.array([2, 3])
print("<== GC ==>")
gc = gradienteConjugado(x0, b)
# print(gc)
print("<== Newton ==>")
print(newtonNDim(x0))

print("<== gradiente fin ==>")
print(grad(gc))

print("<==Hessiano fin ==>")
print(np.linalg.det(hessiano(gc)))





<== GC ==>
[0 0]
x0, f(x^k), aMD, b
[-0.21311475 -0.31967213] 2.307377049180328 0.10655737704918032 0.048978769148078465
[-0.07692308 -0.46153846] 2.230769230769231 0.2406311637080868 3.871678047282085e-31
<== Newton ==>
1 2.230769230769231 [-0.07692308 -0.46153846]
[-0.07692308 -0.46153846]
<== gradiente fin ==>
[0. 0.]
<==Hessiano fin ==>
38.99999999999999


In [8]:
Nfeval = 1


def callbackF(Xi):
    global Nfeval
    print(Nfeval, Xi, f(Xi))
    Nfeval += 1


optimize.minimize(f, x0, method='CG', callback=callbackF,
                  options={"disp": True})


1 [-0.21311475 -0.31967213] 2.3073770505859468
2 [-0.07692308 -0.46153846] 2.230769230769231
Optimization terminated successfully.
         Current function value: 2.230769
         Iterations: 2
         Function evaluations: 15
         Gradient evaluations: 5


     fun: 2.230769230769231
     jac: array([5.96046448e-08, 5.96046448e-08])
 message: 'Optimization terminated successfully.'
    nfev: 15
     nit: 2
    njev: 5
  status: 0
 success: True
       x: array([-0.07692308, -0.46153846])

In [15]:
def grad(x):
    return np.array([
        x[0]**2 + x[1]**2 - 2,
        np.e**(x[0]-1) + x[1]**3 - 2
    ])
def hessiano(x):
    return np.array(
        [ 
            [2*x[0], 2*x[1]],
            [np.e**(x[0]-1), 3*x[1]**2]
        ]
    )

In [62]:
#Calculos de phi
def phiAlpha(x0, alpha, p):
    paX = x0 + p * alpha
    return f(paX)


def phipAlpha(x0, alpha, p):
    x = x0 + alpha * p
    vgrad = grad(x)
    return (np.dot(vgrad, p))


def phipp(x0, alpha, p):
    x = x0 + alpha * p
    ahess = hessiano(x)
    return np.dot(np.dot(ahess, p), p)



# Gradiente Conjugado NL
def newton(xo, p, ao, itmax=100, tol=1e-12):
    k = 0
    # ak = 0
    while abs(phipAlpha(xo, ao, p)) > tol:
        phiap = phipAlpha(xo, ao, p)
        phiapp = phipp(xo, ao, p)
        ak = ao - phiap/phiapp
        # print(f"\t {k}, {ak}")
        k = k+1
        ao = ak
        if k >= itmax:
            print("Iteraciones exedidas")
            break
    return ak


def gradientesConjugados(x0, flavor="FR", k=0, tol=1e-6):
    beta = 0
    p = -grad(x0)
    print("k, fx, x0, alpha, beta, ")
    while np.linalg.norm(grad(x0)) >= tol:
        alpha = newton(x0, p, 0)
        xi = x0 + alpha*p
        gradxi = grad(xi)
        gradx0 = grad(x0)
        if flavor == "FR":
            beta = np.dot(gradxi, gradxi)/np.dot(gradx0, gradx0)
        elif flavor == "PR":
            beta = np.dot(gradxi, (gradxi - gradx0)) / \
                (np.linalg.norm(gradx0))**2
        elif flavor == "PR+":
            beta = np.dot(gradxi, (gradxi - gradx0)) / \
                (np.linalg.norm(gradx0))**2
            if beta < 0:
                beta = 0
        elif flavor == "HS":
            beta = np.dot(gradxi, (gradxi - gradx0)) / \
                np.dot((gradxi - gradx0), p)
        p = -gradxi + beta*p
        print(k,
        #  f(x0),
          xi, alpha, beta)
        x0 = xi
        k += 1
    return x0


def bfgs(x0, tol=1e-6):
    k = 0
    a = np.identity(x0.size)
    g0 = grad(x0)
    # a = jHessFull(x0)
    while (np.linalg.norm(grad(x0)) >= tol):
        s = -np.dot(-grad(x0), a)
        x1 = x0 - s
        g1 = grad(x1)
        y = g1 - g0
        aux = s.T.dot(a).dot(s)
        a = a + (np.dot(y, y)/np.dot(y, s)) - \
            np.dot(np.dot(a, s), np.dot(a, s))/np.dot(np.dot(a, s), s)
        x0 = x1
        print(k, x0, f(x0))
        k = k+1
        # print(a)
    return x0



In [63]:
x0 = np.array([2,3])

bf = bfgs(x0)
pr = gradientesConjugados(x0,"PR+")

0 [ -9.         -24.71828183] 2735.2191335364028
1 [-8043437.13733883 -8027658.15375513] 645827261219418.0
2 [-3.33384719e+34 -3.33384719e+34] 1.1114537077841058e+70
3 [-4.11839926e+172 -4.11839926e+172] inf
4 [nan nan] nan
k, fx, x0, alpha, beta, 
0 [1.16878298 0.90546295] 0.07556518330183765 4.498883645822914e-05
1 [1.01861407 0.96389471] 0.8055766581685698 0.20810485070608264
2 [1.01694439 0.99470223] 0.30575617486552314 0.17226544108783431
3 [0.99987075 1.00588408] 0.6954884681462956 0.2663491661019688
4 [0.99568251 1.00279326] 0.23161278390771853 0
5 [0.99816642 0.99942742] 0.8219354351973654 1.3788992280672818
6 [1.0008882  0.99879124] 0.30323860078273585 0
7 [1.00106525 0.99954874] 0.27713782976479295 0.20228402838725612
8 [1.00007246 1.00030703] 0.9024585359125166 0.5749135201196994
9 [0.99975186 1.00018935] 0.2303925144914516 0
10 [0.99981332 1.0000219 ] 0.5231847234220556 1.059960043444634
11 [1.00003024 0.99991761] 0.4777617357290892 0
12 [1.00005642 0.99997206] 0.2510336451

C:\Users\Daniel\AppData\Local\Temp\ipykernel_13052\3343837133.py:3: RuntimeWarning: overflow encountered in double_scalars
  x[0]**2 + x[1]**2 - 2,
C:\Users\Daniel\AppData\Local\Temp\ipykernel_13052\3343837133.py:4: RuntimeWarning: overflow encountered in double_scalars
  np.e**(x[0]-1) + x[1]**3 - 2
C:\Users\Daniel\AppData\Local\Temp\ipykernel_13052\3996395181.py:80: RuntimeWarning: invalid value encountered in double_scalars
  np.dot(np.dot(a, s), np.dot(a, s))/np.dot(np.dot(a, s), s)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_13052\4236520060.py:2: RuntimeWarning: overflow encountered in double_scalars
  return 4*x[0]**2 + 3*x[0]*x[1] + 3*x[1]**2 + 2*x[0] + 3*x[1] + 3


In [37]:
a = np.linalg.norm(grad(bf))
b = np.linalg.norm(grad(pr))
print(a)
print(b)
np.amin([a,b])


9.344172934156684e-07
7.959922133913181e-07


7.959922133913181e-07

In [22]:
# def grad(x):
#     return np.array([
#         x[0]**2 + x[1]**2 - 2,
#         np.e**(x[0]-1) + x[1]**3 - 2
#     ])


# def hessiano(x):
#     return np.array(
#         [
#             [2*x[0], 2*x[1]],
#             [np.e**(x[0]-1), 3*x[1]**2]
#         ]
#     )

def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2


def fprime(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))



def bfgs(x0, grad, hessian=None, maxiter=100, epsilon=1e-5):
    x = x0
    n = len(x0)
    
    if hessian is None:
        hessian = np.eye(n)
    for i in range(maxiter):
        print(i, x)
        gradient = grad(x)
        if np.linalg.norm(gradient) < epsilon:
            break
        p = -np.dot(hessian, gradient)
        x_new = x + p
        s = x_new - x
        y = grad(x_new) - gradient
        rho = 1.0 / np.dot(y, s)
        hessian = (np.eye(n) - rho * np.outer(s, y)) @ hessian @ (np.eye(n) -
                                                                  rho * np.outer(y, s)) + rho * np.outer(s, s)
        x = x_new
    return x


initial_guess = np.array([2, 2])

result = bfgs(initial_guess, fprime)
print(result)

0 [2 2]
1 [-15.   6.]
2 [1.81562355 6.8935277 ]
3 [ 2.1401269 -1.7359996]
4 [1.92746611 3.80225459]
5 [1.92884475 3.73705796]
6 [1.92799524 3.72623921]
7 [1.91214596 3.60462262]
8 [1.87730953 3.40868998]
9 [1.78078196 2.95674083]
10 [1.64321543 2.41843247]
11 [1.46734561 1.8778594 ]
12 [1.28985069 1.53747275]
13 [1.15504206 1.29320019]
14 [1.03967453 1.04421659]
15 [1.01397751 1.04530299]
16 [0.9976235  0.99405575]
17 [1.00005166 1.00016609]
18 [0.99999891 0.9999994 ]
[0.99999891 0.9999994 ]
